# Convolution base classification

Convolution based classifiers are based on The ROCKET transform [1] and the
subsequent extensions MiniROCKET [2] and MultiROCKET [3]. These transforms can be
used in pipelines, but we provide two convolution based classifiers based on Rocket
for ease of use and reproducability. The RocketClassifier combines the transform with
 a scikitlearn RidgeClassifierCV classifier.  Ther term convolution and kernel are
 used interchangably in this work.

In [5]:
from sktime.registry import all_estimators

# search for all classifiers of type "kernel". This will give some
# UserWarnings if soft dependencies are not installed.
all_estimators(filter_tags={"classifier_type": "kernel"}, estimator_types="classifier")

[('Arsenal', sktime.classification.convolution_based._arsenal.Arsenal),
 ('RocketClassifier',
  sktime.classification.convolution_based._rocket_classifier.RocketClassifier)]

In [9]:
from sklearn.metrics import accuracy_score

from sktime.classification.convolution_based import Arsenal, RocketClassifier
from sktime.datasets import load_basic_motions  # multivariate dataset
from sktime.datasets import load_italy_power_demand  # univariate dataset

italy, italy_labels = load_italy_power_demand(split="train")
italy_test, italy_test_labels = load_italy_power_demand(split="test")
motions, motions_labels = load_basic_motions(split="train")
motions_test, motions_test_labels = load_basic_motions(split="train")
italy.shape

(67, 1, 24)

ROCKET [1] is a transformation that uses random convolutional kernels (random length,
weights, bias, dilation, and padding). ROCKET computes two features from the
resulting feature maps: the max, and the proportion of positive values (or ppv). The
transformed features are used to train a linear classifier. ROCKET compiles (via
Numba) on import, which may take a few seconds. This
notebook is a high level introduction to using and configuring convolution based
classifiers. Details on the algorithm are given in the related [transformation
notebook]().


In [18]:
rocket = RocketClassifier()
rocket.fit(italy, italy_labels)
y_pred = rocket.predict(italy_test)
accuracy_score(italy_test_labels, y_pred)

0.9689018464528668

In [19]:
afc = Arsenal()
afc.fit(italy, italy_labels)
y_pred = afc.predict(italy_test)
accuracy_score(italy_test_labels, y_pred)

0.9698736637512148

RocketClassifier and Arsenal can be configured to use MiniROCKET, MultiROCKET. Simply
 set with rocket_transform as "minirocket" or "multirocket". Both work on multivariate

In [17]:
multi_r = Arsenal(rocket_transform="multirocket")
multi_r.fit(italy, italy_labels)
y_pred = multi_r.predict(italy_test)
accuracy_score(italy_test_labels, y_pred)

0.9698736637512148

0.9708454810495627

0.9708454810495627

In [14]:
mini_r = RocketClassifier(rocket_transform="minirocket")
mini_r.fit(motions, motions_labels)
y_pred = mini_r.predict(motions_test)
accuracy_score(motions_test_labels, y_pred)

1.0

RocketClassifier has three other parameters that may effect performance.
`num_kernels` (default 10,000) determines the number of convolutions/kernels generated
 and will influence the memory usage. `max_dilations_per_kernel` (default=32) and
`n_features_per_kernel` (default=4) are used in 'MiniROCKET' and 'MultiROCKET. For
each candidate convolution, `max_dilations_per_kernel` are assessed and
`n_features_per_kernel` are retained.


## References

[1] Dempster A, Petitjean F and Webb GI (2019) ROCKET: Exceptionally fast
and accurate
time series classification using random convolutional kernels. [arXiv:1910.13051]
(https://arxiv.org/abs/1910.13051, [Journal Paper](https://link.springer.com/article/10
.1007/s10618-020-00701-z)
[2]  Dempster A, Schmidt D and Webb G (2021) MINIROCKET: A Very Fast (Almost)
Deterministic Transform for Time Series Classification [arXiv:2012.08791](https://arxiv
.org/abs/2012.08791) [Conference Paper](https://dl.acm.org/doi/abs/10.1145/3447548.3467231)
[3] Cahng Wei T, Dempster A, Bergmeir C and Webb G (2022) MultiRocket: multiple pooling
operators and transformations for fast and effective time series classification
[Journal Paper](https://link.springer.com/article/10.1007/s10618-022-00844-1)
